In [1]:
#!uv pip install datasets
#uv pip install transformers[torch]

In [3]:
from datasets import load_dataset

dataset = load_dataset("yelp_polarity")  #1
print(dataset)  #2

/home/michele/dev/python/LLMs/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|█| 560000/560000 [00:01<00:00, 435372.82 examples/s
Generating test split: 100%|███| 38000/38000 [00:00<00:00, 566963.17 examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 38000
    })
})


In [4]:
train_dataset = dataset['train']  #1
print(train_dataset[0])  #2

{'text': "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars.", 'label': 0}


In [5]:
train_dataset = dataset["train"]  #1
test_dataset = dataset["test"]   #1

restaurant_train_reviews = train_dataset.filter( #2
    lambda x: "restaurant" in x["text"].lower()
)

restaurant_test_reviews = test_dataset.filter(  #2
    lambda x: "restaurant" in x["text"].lower()
)

number_of_reviews = 5000
subset_train_reviews = restaurant_train_reviews.shuffle(  #3
    seed = 42).select(range(number_of_reviews))
subset_test_reviews = restaurant_test_reviews.shuffle(  #3
    seed = 42).select(range(number_of_reviews))

subset_dataset = {  #4
    "train": subset_train_reviews,
    "test": subset_test_reviews
}

from datasets import DatasetDict
yelp_restaurant_dataset = DatasetDict(subset_dataset)  #5

print(yelp_restaurant_dataset)  #6

Filter: 100%|██████████████████| 38000/38000 [00:00<00:00, 355112.91 examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})


In [6]:
yelp_restaurant_dataset['train'][0]

{'text': 'My girlfriend and I have been wanting to come here for awhile, we finally came & we had the worst experience ever. We asked our server for a few minutes to look over the menu & he never came back. 15 minutes later, someone finally came and took our order. We waited awhile and when they brought our food, they got the whole order wrong. My girlfriend ordered soup and it never came out. Worst service ever. Would not recommend this restaurant to anyone.',
 'label': 0}

In [7]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)  #1

def tokenize_function(examples):  #2
    return tokenizer(examples["text"],
                     padding = "max_length",
                     truncation = True,
                     max_length = 512)

tokenized_datasets = yelp_restaurant_dataset.map(  #3
                         tokenize_function,
                         batched=True)
tokenized_datasets

Map: 100%|█████████████████████████| 5000/5000 [00:01<00:00, 4836.52 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
})

In [8]:
from transformers import AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained(  #1
            model_checkpoint, num_labels = 2)

if torch.backends.mps.is_available():  #2
    device = torch.device("mps")
else:
    device = torch.device(
        "cuda" if torch.cuda.is_available() else "cpu")

model.to(device)  #3

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [11]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(  #1
    output_dir = "./results",  #2
    eval_strategy = "epoch",  #3
    save_strategy = "epoch",  #4
    learning_rate = 2e-5,  #5
    per_device_train_batch_size = 16,  #6
    per_device_eval_batch_size = 16,  #7
    num_train_epochs = 3,  #8
    weight_decay = 0.01,  #9
    logging_dir = "./logs",  #10
    logging_steps = 10,  #11
    save_steps = 500,  #12
    load_best_model_at_end = True,  #13
)

trainer = Trainer(  #14
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["test"],
)

trainer.train()  #15

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`